# UHK SMAP ovalek 01

In [65]:
# install packages
%pip install numpy pandas tensorflow tensorflow_datasets scikit-learn scipy matplotlib

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [66]:
import numpy as np
import pandas as pd
import os
print(os.listdir("./data"))

['small', 'photos_sanitized.csv', 'parameters_products_color_sanitized.csv', 'tmp_dataset', 'parameters_list_sanitized.csv', 'parameters_products_pattern_sanitized.csv']


## Load data

In [67]:
import csv
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf

# load labels
label_ids = []
label_names = []
with open('./data/parameters_list_sanitized.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            line_count += 1
            continue
        else:
            line_count += 1
            if not row[0] in label_ids:
                label_ids += [row[0]]
                label_names += [row[2]]

# load product label relations
products_labels = {}
with open('./data/parameters_products_color_sanitized.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            line_count += 1
            continue
        else:
            line_count += 1
            if not row[0] in products_labels:
                products_labels[row[0]] = []
            for label_index, label_id in enumerate(label_ids):
                if label_id == row[2]:
                    products_labels[row[0]] += [label_names[label_index]]
with open('./data/parameters_products_pattern_sanitized.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            line_count += 1
            continue
        else:
            line_count += 1
            if not row[0] in products_labels:
                products_labels[row[0]] = []
            for label_index, label_id in enumerate(label_ids):
                if label_id == row[2]:
                    products_labels[row[0]] += [label_names[label_index]]

# load products with main photo
products = []
with open('./data/photos_sanitized.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
            line_count += 1
            if row[4] != 'Y':
                continue
            found = False
            for product in products:
                if product[0] == row[0]:
                    found = True
                    break
            if not found:
                products += [(row[0], row[1], row[3], row[4], ), ]
    print(f'Processed {line_count} lines.')
    print(f'Found products {products.__len__}.')


Column names are id_product, id_photo, date_update, position, show_in_lead
Processed 16161 lines.
Found products <method-wrapper '__len__' of list object at 0x7f83cf3913c0>.


In [68]:
from os.path import exists

# prepare data for dataset
data_photos_paths = []
data_labels = []
for product in products:
    tmp_path = f'./data/small/{int(product[0]) % 10}/{product[0]}/{product[0]}_{product[1]}_{product[3]}_{product[2]}_small.webp'
    if not exists(tmp_path):
        continue
    data_photos_paths += [tmp_path]
    tmp_product_labels = products_labels[product[0]]
    tmp_labels_vector = []
    for label in label_names:
        tmp_labels_vector += [1 if label in tmp_product_labels else 0]
    data_labels += [tmp_labels_vector]

## Prepare train test and validation data

In [69]:
from sklearn.model_selection import train_test_split

data_photos_paths_train, data_photos_paths_tmp, data_labels_train, data_labels_tmp = train_test_split(data_photos_paths, data_labels, test_size=0.25, random_state=42)
data_photos_paths_test, data_photos_paths_validation, data_labels_test, data_labels_validation = train_test_split(data_photos_paths_tmp, data_labels_tmp, test_size=0.4, random_state=None, shuffle=False)



def _create_dataset(X, y):
    # X = tf.constant(X)
    # y = tf.constant(y)
    dataset = tf.data.Dataset.from_tensor_slices((X, y))

    def _prepare_picture_func(picturepath, labels):
        image_string = tf.io.read_file(picturepath)
        image = tf.image.decode_image(image_string, channels=3)
        image.set_shape([None, None, None])
        image = tf.image.resize(image, [250, 340])
        # image = tf.subtract(image, 116.779) # Zero-center by mean pixel
        # # image.set_shape([200, 200, 3])
        # image.set_shape((250, 250, 3))
        image.set_shape((250, 340, 3))
        # image = image.reshape(-1, 250, 340, 3)
        # image = tf.reverse(image, axis=[2]) # 'RGB'->'BGR'
        # d = dict(zip([picturepath], [image])), labels
        # return (picturepath, image), labels
        return image, labels
        # return d
    return dataset.map(_prepare_picture_func)

train_ds = _create_dataset(data_photos_paths_train, data_labels_train)
test_ds = _create_dataset(data_photos_paths_test, data_labels_test)
validation_ds = _create_dataset(data_photos_paths_validation, data_labels_validation)


# data_photos_paths = tf.constant(data_photos_paths)
# data_labels = tf.constant(data_labels)
# dataset = tf.data.Dataset.from_tensor_slices((data_photos_paths, data_labels))

# def _prepare_picture_func(picturepath, labels):
#     image_string = tf.io.read_file(picturepath)
#     image = tf.image.decode_image(image_string, channels=3)
#     image.set_shape([None, None, None])
#     image = tf.image.resize(image, [150, 150])
#     image = tf.subtract(image, 116.779) # Zero-center by mean pixel
#     image.set_shape([150, 150, 3])
#     # image = tf.reverse(image, axis=[2]) # 'RGB'->'BGR'
#     # d = dict(zip([picturepath], [image])), labels
#     return (picturepath, image), labels
#     # return d
# dataset = dataset.map(_prepare_picture_func)



# import tensorflow_datasets as tfds

# dataset = dataset.shuffle(buffer_size=100, seed=19575364)
# dataset.save('./data/tmp_dataset')



# train_set, test_set, valid_set = tfds.load('./data/tmp_dataset', 
#                                            split=["train[0%:75%]", "test[75%:90%]", "valid[90%:100%]"],
#                                            as_supervised=True, with_info=True, download=False)

#batch = dataset.batch(20)


# train_set, tmp_set = tf.keras.utils.split_dataset(
#     dataset, left_size=0.75, right_size=0.25, shuffle=True, seed=19575364
# )
# test_set, valid_set = tf.keras.utils.split_dataset(
#     dataset, left_size=0.60, right_size=0.40, shuffle=False, seed=None
# )


In [70]:
# train_batch = train_ds.batch(32)
# test_batch = test_ds.batch(32)

# # check batch
# for image_batch, labels_batch in train_batch:
#   print(image_batch.shape)
#   print(labels_batch.shape)
#   break


## Create NN model

In [71]:
# Disable warnings
import warnings
warnings.filterwarnings('ignore')

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf
tf.compat.v1.enable_eager_execution()


# # Initializes a sequential model
# model = Sequential()

# # First layer (pictures are 240x340=85000 x 3 colors => 255000)
# # model.add(Dense(2048, activation='relu', input_shape=(255000, )))
# model.add(Dense(2048, activation='relu', input_shape=(250, 250, 3)))
# # model.add(Dense(2048, activation='relu', input_shape=(187500, )))
# # model.add(Dense(2048, activation='relu', input_dim=187500))

# # Second layer
# model.add(Dense(256, activation='relu'))

# # Output layer (multi-label classification therefore use sigmoid activation function)
# model.add(Dense(48, activation='sigmoid'))

# model.summary()


num_classes = len(label_names)

model = Sequential([
  layers.Rescaling(1./255, input_shape=(250, 340, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='sigmoid'),
  layers.Dense(num_classes)
])

## Compile NN

In [75]:
# multi-label classification therefore use binary_crossentropy
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'], run_eagerly=True)

# model.compile(optimizer='adam',
#               loss=tf.keras.losses.binary_crossentropy(from_logits=True),
#               metrics=['accuracy'])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'],
              run_eagerly=True)

model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_5 (Rescaling)     (None, 250, 340, 3)       0         
                                                                 
 conv2d_15 (Conv2D)          (None, 250, 340, 16)      448       
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 125, 170, 16)     0         
 g2D)                                                            
                                                                 
 conv2d_16 (Conv2D)          (None, 125, 170, 32)      4640      
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 62, 85, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 62, 85, 64)       

## Use NN for data

In [73]:

# PIL.Image.open(str(data_photos_paths[58]))

# import matplotlib.pyplot as plt

# @tf.function
# def _preview():
#   plt.figure(figsize=(10, 10))
#   for images, labels in train_ds.take(1):
#     for i in range(9):
#       ax = plt.subplot(3, 3, i + 1)
#       # plt.imshow(images[i].numpy().astype("uint8"))
#       # plt.imshow(images[i][1])
#       tmp_title = []
#       for label_index in labels:
#         tf.gather(list, tf_look_up[index])
#         tmp_title += [label_names[label_index]]
#       plt.title(', '.join(tmp_title))
#       plt.axis("off")
# _preview()



# import matplotlib.pyplot as plt

# @tf.function
# def preview():
#     plt.figure(figsize=(10, 10))
#     for images, labels in train_ds.take(1):
#         for i in range(9):
#             ax = plt.subplot(3, 3, i + 1)
#             plt.imshow(images[i].numpy().astype("uint8"))
#             plt.title(labels[labels[i]])
#             plt.axis("off")


In [76]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

callbacks = [
    EarlyStopping(patience=10, verbose=1),
    ReduceLROnPlateau(factor=0.1, patience=3, min_lr=0.00001, verbose=1),
    ModelCheckpoint('model.h5', verbose=1, save_best_only=True, save_weights_only=True)
]

# model.fit_generator(
#         train_batch, 
#         steps_per_epoch=len(train_batch), # batch_size
#         validation_data=test_batch,
#         validation_steps=len(test_batch), # batch_size
#         epochs=10,
#         callbacks=callbacks
#     )
# epochs=10
# history = model.fit(
#   train_ds,
#   validation_data=validation_ds,
#   epochs=epochs
# )
epochs=10
history = model.fit(
  train_ds.batch(32),
  validation_data=validation_ds.batch(32),
  epochs=epochs
)




Epoch 1/10
  4/181 [..............................] - ETA: 2:12 - loss: 0.5254 - accuracy: 0.0000e+00

InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Input is empty.
	 [[{{node decode_image/DecodeImage}}]] [Op:IteratorGetNext]